In [6]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils import data
import numpy as np

In [7]:
T = 1000  # 总共产生1000个点
time = torch.arange(1, T + 1, dtype=torch.float32)
x = torch.sin(0.01 * time) + torch.normal(0, 0.2, (T,)) # sin函数

tau = 4 # 时间维度/序列长度
features = torch.zeros((T - tau, tau, 1))  # （batch, seq, feature）
for i in range(tau):
    features[:, i] = x[i: T - tau + i].unsqueeze(1)

labels = x[tau:]    
batch_size, n_train = 16, 600 # 批量大小、训练样本数量
# 只有前n_train个样本用于训练
data_arrays = (features[:n_train], labels[:n_train])
dataset = data.TensorDataset(*data_arrays)
train_iter = data.DataLoader(dataset, batch_size, shuffle=True) # 打乱顺序

# 定义模型并训练

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 是否采用GPU

In [8]:
# 定义模型
class GRU_REG(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2, seq=4):
        super(GRU_REG, self).__init__()
        
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers) # rnn
        
        for name, param in self.gru.named_parameters():
            if name.startswith("weight"):
                nn.init.xavier_normal_(param)
            else:
                nn.init.zeros_(param)

        self.reg = nn.Linear(hidden_size * seq, output_size) # 回归
        
    def forward(self, x):
        x, _ = self.gru(x) # (seq, batch, hidden)
        x = torch.transpose(x, 1, 0) # 调整为 (batch, seq, hidden)
        b, s, h = x.shape
        x = x.reshape(b, s*h) # 转换成线性层的输入格式
        x = self.reg(x)
        return x

# 模型的初始化
model = GRU_REG(input_size=1, hidden_size=2, output_size=1, num_layers=2, seq=tau).to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)


# 模型的训练
num_epochs = 5
hist = np.zeros(num_epochs) # 用来记录每一个epoch的误差

In [9]:
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for num, (X, y) in enumerate(train_iter):
        X, y = X.to(device), y.to(device)
        X = torch.transpose(X, 1, 0) # 将batch和seq的维度置换一下
        y = y.reshape(-1, 1)
        optimizer.zero_grad()
        
        # forward
        output = model(X)
        
        # backward
        loss = criterion(output, y)
        loss.backward()
        # optimize
        optimizer.step()
        
    with torch.no_grad():
        epoch_loss += loss.detach().cpu().item() # 由于我是在GPU进行训练的，因此这里将计算出的损失脱离GPU
    print(f'epoch {epoch + 1}, loss {epoch_loss:f}') 

epoch 1, loss 0.435268
epoch 2, loss 0.248544
epoch 3, loss 0.320258
epoch 4, loss 0.171828
epoch 5, loss 0.036388
